## 1.1 Import Libraries

In [1]:
import numpy as np
from random import uniform

## 1.2 Load and Process Data

In [2]:
filename = 'alice.txt'
data = open(filename, encoding="UTF-8").read()

In [3]:
char =  list(set(data))   ## Number of Different Chracters in Library
vocab_length = len(char)  ## Length of the vocabulary
data_length =  len(data)  ## Length of the data

### 1.2.1 Mapping characters in vocabulary to Integer Index and vice_versa

In [4]:
char_int = {ch : i for i,ch in enumerate(char)}
int_char = {i:ch for i,ch in enumerate(char)}


## 1.3 Model Paramters

In [5]:
hidden = 128 # Size of Hidden Layer 
sequence= 25 # Number of steps to unroll the RNN 

### Wax,Was,Way,b1,b2 are the parameters of RNN. Equation for forward pass is below:
$$ s = tanh(W_{ax}.x + W_{as}.s_{t-1} + b1) $$
$$ y = softmax(W_{ay}.s{t} + b2) $$

In [6]:
Wax = np.random.randn(hidden, vocab_length) * 0.01
Was = np.random.randn(hidden,hidden) * 0.01
Way = np.random.randn(vocab_length,hidden) * 0.01
b1 = np.zeros((hidden,1))
b2 = np.zeros((vocab_length,1))
initial_hidden = np.zeros((hidden,1))

## 1.4 Training and Generation 

In [7]:
def calculateLoss(inputs, labels, initial_hidden):
    '''
    This function calculates the loss and performs 
    backpropagation in time. The loss function is 
    cross-entropy loss function. It also returns the 
    gradients of RNN paramters which is to be used 
    by optimizer.
    

    Parameters
    ----------
    inputs : List of Integers. Each element in list corresponds to a character's 
             integer value, which is determined by char_int dictionary. 

    labels : True Labels corresponding to inputs.It is next character 
             to input in training data.
             It is also List of Integers where integer mapping is determined as for inputs.
             
    initial_hidden: An initial state given to RNN.
             
    Returns
    -------
    loss   : The cross entropy loss on inputs
    dwax   : Gradient of loss wrt parameter Wax.
    dwas   : Gradient of loss wrt parameter Was.
    dway   : Gradient of loss wrt parameter Way.
    db1    : Gradient of loss wrt parameter b1.
    db2    : Gradient of loss wrt parameter b2.
    
    s[len(inputs)-1] : 
    
    '''
    
    ## These dictionaries hold various activation at time ##
    
    x = {}  ##  Holds input characters as one hot vectors  
    s = {}  ## Ouput Activation at Layer 1 
    y = {} ## Output from softmax 
    a1 ={} ## Input Preactivation at Layer 1 
    a2= {}  ## Input Preactivation at Layer 2
    
    s[-1] = initial_hidden
    loss = 0
    
    ## Forward Propagate 
    
    for t in range(len(inputs)):
        
        ## Encode to one hot vector to give x
        ## dimensions of x[t] = vocab_length * 1
        x[t] = np.zeros((vocab_length,1))
        x[t][inputs[t]] = 1
        
        ## Calculate preactivation at Layer 1 (Input Layer)
        ## dimensions of a1 = (hidden,1)
        a1[t] =  np.dot(Wax, x[t]) + np.dot(Was,s[t-1]) + b1
        
        ## Calculate activation at Layer 1
        ## dimensions of s1 =  (hidden,1)
        s[t]  =  np.tanh(a1[t])    
        
        ## Input Layer Preactivation at Layer 2
        ## dimensions of a2 =  (vocab_length,1)
        a2[t] =  np.dot(Way, s[t]) + b2
        
        ## Output of RNN (softmax function)
        ## dimensions of y =  (vocab_length,1)
        y[t] =   np.exp(a2[t])/np.sum(np.exp(a2[t]))
        
        ## Cross Entropy Loss
        loss += -np.log(y[t][labels[t],0]) 
        
    ### Backpropagation in Time
    
    #
    dwas = np.zeros_like(Was) 
    dway = np.zeros_like(Way)
    dwax = np.zeros_like(Wax)
    db1 = np.zeros_like(b1)
    db2 = np.zeros_like(b2)
    dsnext = np.zeros_like(s[0]) ## This is the derivative component for s from layer ahead in time.
    
    for t in range(len(inputs)-1,-1,-1):
        ## Converted true label to one hot vector
        e = np.zeros_like(y[0])
        e[labels[t]] = 1
        
        ## Backpropgate through softmax
        e = (y[t]-e)
        
        ## Compute gradients for Way and b2
        dway +=  np.dot(e, s[t].T)
        db2  +=  e
        
        ## Calculate Gradient of output activation
        ## of layer 1 (s). Also, add the gradient
        ## coming from previous step.
        ds = np.dot(Way.T, e) + dsnext
        
        ## Backpropgate into tanh non-linearlity
        ## Gradient wrt to input preactivation at layer 1
        da1 = (1-s[t]*s[t])*ds
        
        ### Gradient wrt to Wax, Was, b1
        dwax +=  np.dot(da1, x[t].T)
        db1  +=  da1
        dwas += np.dot(da1, s[t-1].T)
        
        ### Calculate gradient wrt to s{t-1} at time step
        ## t. This will be added to s{t-1} at time step t-1
        ## More evident from drawing computation graph.
        dsnext =  np.dot(Was.T, da1)
    
    ### Mitigating the exploding gradients problem
    ### See Pascanu, R., Mikolov, T., & Bengio, Y. (2013, February). 
    ## On the difficulty of training recurrent neural networks(ICML).
    for dparam in [dwax,dwas,dway,db1,db2]:
        np.clip(dparam, -5, 5, out=dparam) 
  
    return loss,dwax,dwas,dway,db1,db2,s[len(inputs)-1]
        
        

In [8]:
def sample(s, seed_ix, n):
    """
        Sample a sequence of integers from the model.
        Runs the RNN in forward mode for n steps. Returns
        the sequence of characters generated by RNN
        
        Parameters
        -------------
        n       : The number of steps to run RNN for
        seed_ix : The initial seed to start generating characters from
        s       : Initial memory state
        
        Returns
        -------------
        char_seq  : An array of Integers, which corresponds to character sequence
                  generated by RNN. The characters can be retrieved from 
                  int_char dictionary defined earlier in data processing section
        
    """
    x = np.zeros((vocab_length, 1))
    x[seed_ix] = 1
    char_seq = []

    for t in range(n):
        # Run the forward pass only.
        s = np.tanh(np.dot(Wax, x) + np.dot(Was, s) + b1)
        a2 = np.dot(Way, s) + b2
        y = np.exp(a2) / np.sum(np.exp(a2))

        # Sample from the distribution produced by softmax.
        ix = np.random.choice(range(vocab_length), p=y.ravel())

        # Prepare input for the next cell.
        x = np.zeros((vocab_length, 1))
        x[ix] = 1
        char_seq.append(ix)
    return char_seq

In [9]:
def gradCheck(inputs, targets, hprev):
  global Wax, Was, Way, b1, b2
  num_checks, delta = 10, 1e-5
  _, dwax,dwas,dway,db1,db2,_ = calculateLoss(inputs, targets, hprev)
  for param,dparam,name in zip([ Wax, Was, Way, b1, b2],
                               [dwax,dwas,dway,db1,db2],
                               ['Wax', 'Was', 'Way', 'b1', 'b2']):
    s0 = dparam.shape
    s1 = param.shape
    assert s0 == s1, 'Error dims dont match: %s and %s.' % (s0, s1)
    print(name)
    for i in range(num_checks):
      ri = int(uniform(0,param.size))
      # evaluate cost at [x + delta] and [x - delta]
      old_val = param.flat[ri]
      param.flat[ri] = old_val + delta
      cg0, _, _, _, _, _,_ = calculateLoss(inputs, targets, hprev)
      param.flat[ri] = old_val - delta
      cg1, _, _, _, _, _,_ = calculateLoss(inputs, targets, hprev)
      param.flat[ri] = old_val # reset old value for this parameter
      # fetch both numerical and analytic gradient
      grad_analytic = dparam.flat[ri]
      grad_numerical = (cg0 - cg1) / ( 2 * delta )
      rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
      print('%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error))
      # rel_error should be on order of 1e-7 or less

In [10]:
def basicGradCheck():
    inputs = [char_int[ch] for ch in data[:sequence]]
    targets = [char_int[ch] for ch in data[1:sequence+1]]
    hprev = np.zeros((hidden,1)) # reset RNN memory
    gradCheck(inputs, targets, hprev)

In [11]:
basicGradCheck()

Wax
0.000000, 0.000000 => nan 
0.025383, 0.025383 => 1.628963e-08 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 

C:\Users\himan\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars



-0.001834, -0.001834 => 1.880710e-07 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
Was
0.000042, 0.000042 => 1.490677e-05 
0.000705, 0.000705 => 2.742790e-07 
-0.000104, -0.000104 => 4.299940e-07 
-0.000186, -0.000186 => 2.516666e-06 
-0.000283, -0.000283 => 1.690710e-06 
-0.000023, -0.000023 => 2.417268e-05 
-0.000626, -0.000626 => 2.619935e-07 
0.000150, 0.000150 => 2.233340e-06 
0.000699, 0.000699 => 9.612677e-07 
-0.000266, -0.000266 => 4.967051e-07 
Way
-0.000684, -0.000684 => 1.276520e-06 
-0.000058, -0.000058 => 6.601337e-06 
0.000573, 0.000573 => 9.821675e-07 
-0.014387, -0.014387 => 2.236810e-08 
-0.000197, -0.000197 => 3.601451e-06 
0.000014, 0.000014 => 1.654030e-05 
0.000033, 0.000033 => 1.649125e-06 
0.008641, 0.008641 => 9.250453e-09 
-0.000229, -0.000229 => 1.562184e-06 
-0.015704, -0.015704 => 5.711710e-08 
b1
-0.071859, -0.071859 => 5.806325e-09 
-0.043646, -0.043646 => 2.389464e-08 
0.032986, 0.032986 => 7.768636e-09 
0.004281, 0.004281 => 1.823768e-07 
-0.0

## Learning the Weights

In [ ]:
n,p = 0,0

mWax, mWas, mWay = np.zeros_like(Wax), np.zeros_like(Was), np.zeros_like(Way)

mb1, mb2 = np.zeros_like(b1), np.zeros_like(b2)

smooth_loss = -np.log(1.0/vocab_length)*sequence

while p <1000000:
    # Prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+sequence+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden, 1)) # reset RNN memory
        p = 0 # go from start of data

    # In each step we unroll the RNN for seq_length cells, and present it with
    # sequence length inputs and sequence length  target outputs to learn.
    inputs = [char_int[ch] for ch in data[p:p+sequence]]
    targets = [char_int[ch] for ch in data[p+1:p+sequence+1]]

    # Sample from the model now and then.
    if n % 10000 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(int_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # Forward sequence length characters through the net and fetch gradient
    loss, dWax, dWas, dWay, db1, db2,hprev = calculateLoss(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 500 == 0: print('iter %d (p=%d), loss: %f' % (n, p, smooth_loss))

    # Perform parameter update with Adagrad
    for param, dparam, mem in zip([Wax, Was, Way, b1, b2],[dWax, dWas, dWay, db1, db2],[mWax, mWas, mWay,mb1,mb2]):
        mem += dparam * dparam
        param += -0.1 * dparam / np.sqrt(mem + 1e-8)

    p += sequence
    n += 1

----
 o sttonglpet, iPk gelee doike. ‘Hould’l theen’s Tat thr ire veh ig ingh yout ind tut ases the
bercund chisew toimy ange Halget.’

E:’r ars!’

rle, waiky rot, and rreruwedy shel ur thedr C!’ 
Le sald e 
----
iter 0 (p=0), loss: 106.932519
iter 500 (p=12500), loss: 94.465643
iter 1000 (p=25000), loss: 82.885664
iter 1500 (p=37500), loss: 74.838225
iter 2000 (p=50000), loss: 69.646248
iter 2500 (p=62500), loss: 65.651194
iter 3000 (p=75000), loss: 62.760084
iter 3500 (p=87500), loss: 61.152050
iter 4000 (p=100000), loss: 59.678742
iter 4500 (p=112500), loss: 59.171045
iter 5000 (p=125000), loss: 58.906037
iter 5500 (p=137500), loss: 58.080072
iter 6000 (p=5600), loss: 58.091562
iter 6500 (p=18100), loss: 57.399624
iter 7000 (p=30600), loss: 56.919954
iter 7500 (p=43100), loss: 56.850266
iter 8000 (p=55600), loss: 56.287581
iter 8500 (p=68100), loss: 56.018038
iter 9000 (p=80600), loss: 55.346352
iter 9500 (p=93100), loss: 55.233367
----
 she gyoutle fore, in tvi: he.
rforncut to kar

iter 82500 (p=40900), loss: 50.926881
iter 83000 (p=53400), loss: 50.428268
iter 83500 (p=65900), loss: 50.351247
iter 84000 (p=78400), loss: 49.741620
iter 84500 (p=90900), loss: 49.800039
iter 85000 (p=103400), loss: 49.495639
iter 85500 (p=115900), loss: 49.369346
iter 86000 (p=128400), loss: 49.602021
iter 86500 (p=140900), loss: 49.760786
iter 87000 (p=9000), loss: 50.164074
iter 87500 (p=21500), loss: 50.273807
iter 88000 (p=34000), loss: 50.477719
iter 88500 (p=46500), loss: 50.532039
iter 89000 (p=59000), loss: 50.293374
iter 89500 (p=71500), loss: 49.782542
----
 s.

‘It quen,
litthing, to-’

‘Jur wis the bebuthen dou nignink she catch sell, ‘Bhe: for
Hat’r taid Alar coup.

ATines eve wat a seam, a mo, hare trilge sheme Bo,’
soane nowe: ‘Oofthy tooklirmome cea 
----
iter 90000 (p=84000), loss: 49.391592
iter 90500 (p=96500), loss: 49.487886
iter 91000 (p=109000), loss: 49.327503
iter 91500 (p=121500), loss: 49.541688
iter 92000 (p=134000), loss: 49.364653
iter 92500 (p=2100), 